# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [35]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

In [6]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3, os, sys
from sagemaker.pytorch import PyTorch

In [7]:
from sagemaker import get_execution_role
role = get_execution_role()
sess = sagemaker.Session()
sess.default_bucket(), sess.boto_region_name, get_execution_role()

('sagemaker-us-east-1-176302953206',
 'us-east-1',
 'arn:aws:iam::176302953206:role/service-role/AmazonSageMaker-ExecutionRole-20211110T114740')

In [38]:
bucket= 'image-classification-using-aws-sagemaker' 


In [39]:
os.environ["DEFAULT_S3_BUCKET"] = bucket
! echo "$DEFAULT_S3_BUCKET"

image-classification-using-aws-sagemaker


## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [ ]:
#TODO: Fetch and upload the data to AWS S3
# Command to download and unzip data
#!wget -O https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
    
!wget -q https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip -O ./dogImages.zip

!unzip dogImages.zip

In [8]:
! echo s3://${DEFAULT_S3_BUCKET}/valid/

s3://image-classification-using-aws-sagemaker/valid/


In [ ]:
!aws s3 sync ./dogImages/valid  s3://${DEFAULT_S3_BUCKET}/valid/

In [ ]:
!aws s3 sync ./dogImages/test  s3://${DEFAULT_S3_BUCKET}/test/

In [ ]:
!aws s3 sync ./dogImages/train s3://${DEFAULT_S3_BUCKET}/train/

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [40]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

In [41]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "batch_size" : CategoricalParameter([32, 64, 128]),
    "learning_rate" : ContinuousParameter(0.001, 0.01),
}

objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Testing Loss: ([0-9\\.]+)"}]


In [22]:
#TODO: Create estimators for your HPs

# TODO: Your estimator here
estimator = PyTorch(
    entry_point="hpo.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    #instance_type="ml.m5.large"
    instance_type="ml.p3.2xlarge",
)

# TODO: Your HP tuner here
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=6,
    max_parallel_jobs=2,
    objective_type=objective_type)

In [23]:
{"training": f"s3://{bucket}/"}

{'training': 's3://image-classification-using-aws-sagemaker/'}

In [24]:
# TODO: Fit your HP Tuner
tuner.fit({"training": f"s3://{bucket}/"}, wait=True) # TODO: Remember to include your data channels

.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................!


In [25]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()#TODO

#Get the hyperparameters of the best trained model
#hyperparameters = best_estimator.hyperparameters()
#hyperparameters



hyperparameters = {"batch_size": int(best_estimator.hyperparameters()['batch_size'].replace('"', '')), \
                   "learning_rate": best_estimator.hyperparameters()['learning_rate']}
hyperparameters


2022-01-11 21:12:49 Starting - Preparing the instances for training
2022-01-11 21:12:49 Downloading - Downloading input data
2022-01-11 21:12:49 Training - Training image download completed. Training in progress.
2022-01-11 21:12:49 Uploading - Uploading generated training model
2022-01-11 21:12:49 Completed - Training job completed


{'batch_size': 32, 'learning_rate': '0.0025749942716753217'}

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [42]:
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import (
    Rule,
    DebuggerHookConfig,
    rule_configs,
)
from sagemaker.debugger import ProfilerRule, ProfilerConfig, FrameworkProfile


In [43]:
# TODO: Set up debugging and profiling rules and hooks
rules = [
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.exploding_tensor()),    
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.class_imbalance()), #specific to DL frameworks
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.OverallSystemUsage()), 
    ProfilerRule.sagemaker(rule_configs.CPUBottleneck()),    
    #ProfilerRule.sagemaker(rule_configs.OverallFrameworkMetrics()), #specific to DL frameworks   

    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]


hook_config = DebuggerHookConfig(
    hook_parameters={
        "train.save_interval": "100",
        "eval.save_interval": "10"
    }
)

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=2)
)

debugger_config = DebuggerHookConfig(hook_parameters={"train.save_interval":"100", "eval.save_interval":"10"})

In [44]:
os.environ['SM_CHANNEL_TRAINING']=f's3://{bucket}/'
os.environ['SM_MODEL_DIR']=f's3://{bucket}/model/'
os.environ['SM_OUTPUT_DATA_DIR']=f's3://{bucket}/output/'

!echo $SM_CHANNEL_TRAINING
!echo $SM_MODEL_DIR
!echo $SM_OUTPUT_DATA_DIR

s3://image-classification-using-aws-sagemaker/
s3://image-classification-using-aws-sagemaker/model/
s3://image-classification-using-aws-sagemaker/output/


In [45]:
hyperparameters = {'batch_size': 32, 'learning_rate': '0.0025749942716753217'}
hyperparameters

{'batch_size': 32, 'learning_rate': '0.0025749942716753217'}

In [48]:
# TODO: Create and fit an estimator

# TODO: Your estimator here
estimator = PyTorch(
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    entry_point="train_model.py",
    framework_version="1.8",
    py_version="py36",
    hyperparameters=hyperparameters,
    profiler_config=profiler_config,
    debugger_hook_config=debugger_config,
    rules=rules,
)

estimator.fit({"training": f"s3://{bucket}/"}, wait=True)

2022-01-12 14:35:02 Starting - Starting the training job...
2022-01-12 14:35:07 Starting - Launching requested ML instancesLossNotDecreasing: InProgress
VanishingGradient: InProgress
ExplodingTensor: InProgress
Overfit: InProgress
ClassImbalance: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
LowGPUUtilization: InProgress
OverallSystemUsage: InProgress
CPUBottleneck: InProgress
ProfilerReport: InProgress
......
2022-01-12 14:36:25 Starting - Preparing the instances for training.........
2022-01-12 14:37:56 Downloading - Downloading input data............
2022-01-12 14:39:57 Training - Downloading the training image...........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-01-12 14:41:43,476 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-01-12 14:41:43,499 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2

In [49]:
# TODO: Plot a debugging output.

from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

trial = create_trial(estimator.latest_job_debugger_artifacts_path())


[2022-01-12 14:59:09.309 ip-172-16-31-62:15716 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-01-12 14:59:09.342 ip-172-16-31-62:15716 INFO s3_trial.py:42] Loading trial debug-output at path s3://sagemaker-us-east-1-176302953206/pytorch-training-2022-01-12-14-35-02-261/debug-output


In [50]:
trial.tensor_names()

[2022-01-12 15:01:25.946 ip-172-16-31-62:15716 INFO trial.py:198] Training has ended, will refresh one final time in 1 sec.
[2022-01-12 15:01:26.972 ip-172-16-31-62:15716 INFO trial.py:210] Loaded all steps


['CrossEntropyLoss_output_0',
 'gradient/ResNet_fc.0.bias',
 'gradient/ResNet_fc.0.weight',
 'gradient/ResNet_fc.2.bias',
 'gradient/ResNet_fc.2.weight',
 'layer1.0.relu_input_0',
 'layer1.0.relu_input_1',
 'layer1.0.relu_input_2',
 'layer1.1.relu_input_0',
 'layer1.1.relu_input_1',
 'layer1.1.relu_input_2',
 'layer1.2.relu_input_0',
 'layer1.2.relu_input_1',
 'layer1.2.relu_input_2',
 'layer2.0.relu_input_0',
 'layer2.0.relu_input_1',
 'layer2.0.relu_input_2',
 'layer2.1.relu_input_0',
 'layer2.1.relu_input_1',
 'layer2.1.relu_input_2',
 'layer2.2.relu_input_0',
 'layer2.2.relu_input_1',
 'layer2.2.relu_input_2',
 'layer2.3.relu_input_0',
 'layer2.3.relu_input_1',
 'layer2.3.relu_input_2',
 'layer3.0.relu_input_0',
 'layer3.0.relu_input_1',
 'layer3.0.relu_input_2',
 'layer3.1.relu_input_0',
 'layer3.1.relu_input_1',
 'layer3.1.relu_input_2',
 'layer3.2.relu_input_0',
 'layer3.2.relu_input_1',
 'layer3.2.relu_input_2',
 'layer3.3.relu_input_0',
 'layer3.3.relu_input_1',
 'layer3.3.rel

In [51]:
len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.TRAIN))

1

In [52]:
len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.EVAL))

1

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [59]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

training_job_name = estimator.latest_training_job.name
tj = TrainingJob(training_job_name, boto3.session.Session().region_name)
tj.wait_for_sys_profiling_data_to_be_available()

ProfilerConfig:{'S3OutputPath': 's3://sagemaker-us-east-1-176302953206/', 'ProfilingIntervalInMilliseconds': 500, 'ProfilingParameters': {'DataloaderProfilingConfig': '{"StartStep": 0, "NumSteps": 2, "MetricsRegex": ".*", }', 'DetailedProfilingConfig': '{"StartStep": 0, "NumSteps": 2, }', 'FileOpenFailThreshold': '50', 'HorovodProfilingConfig': '{"StartStep": 0, "NumSteps": 2, }', 'LocalPath': '/opt/ml/output/profiler', 'PythonProfilingConfig': '{"StartStep": 0, "NumSteps": 2, "ProfilerName": "cprofile", "cProfileTimer": "total_time", }', 'RotateFileCloseIntervalInSeconds': '60', 'RotateMaxFileSizeInBytes': '10485760', 'SMDataParallelProfilingConfig': '{"StartStep": 0, "NumSteps": 2, }'}}
s3 path:s3://sagemaker-us-east-1-176302953206/pytorch-training-2022-01-12-14-35-02-261/profiler-output


Profiler data from system is available


In [99]:
estimator.latest_training_job.name

'pytorch-training-2022-01-12-14-35-02-261'

In [60]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

[2022-01-12 15:04:41.557 ip-172-16-31-62:15716 INFO metrics_reader_base.py:134] Getting 18 event files
select events:['total']
select dimensions:['CPU', 'GPU']
filtered_events:{'total'}
filtered_dimensions:{'GPUUtilization-nodeid:algo-1', 'CPUUtilization-nodeid:algo-1', 'GPUMemoryUtilization-nodeid:algo-1'}


In [63]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"

In [100]:
rule_output_path

's3://sagemaker-us-east-1-176302953206/pytorch-training-2022-01-12-14-35-02-261/rule-output'

In [64]:
! aws s3 ls {rule_output_path} --recursive

2022-01-12 14:55:05     430351 pytorch-training-2022-01-12-14-35-02-261/rule-output/ProfilerReport/profiler-output/profiler-report.html
2022-01-12 14:55:04     288475 pytorch-training-2022-01-12-14-35-02-261/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb
2022-01-12 14:54:58        546 pytorch-training-2022-01-12-14-35-02-261/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json
2022-01-12 14:54:58      58664 pytorch-training-2022-01-12-14-35-02-261/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
2022-01-12 14:54:58       1936 pytorch-training-2022-01-12-14-35-02-261/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json
2022-01-12 14:54:58        130 pytorch-training-2022-01-12-14-35-02-261/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
2022-01-12 14:54:58      10082 pytorch-training-2022-01-12-14-35-02-261/rule-output/ProfilerReport/profiler-output/profiler-re

In [65]:
! aws s3 cp {rule_output_path} ./ --recursive

download: s3://sagemaker-us-east-1-176302953206/pytorch-training-2022-01-12-14-35-02-261/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json to ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
download: s3://sagemaker-us-east-1-176302953206/pytorch-training-2022-01-12-14-35-02-261/rule-output/ProfilerReport/profiler-output/profiler-reports/LoadBalancing.json to ProfilerReport/profiler-output/profiler-reports/LoadBalancing.json
download: s3://sagemaker-us-east-1-176302953206/pytorch-training-2022-01-12-14-35-02-261/rule-output/ProfilerReport/profiler-output/profiler-reports/IOBottleneck.json to ProfilerReport/profiler-output/profiler-reports/IOBottleneck.json
download: s3://sagemaker-us-east-1-176302953206/pytorch-training-2022-01-12-14-35-02-261/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json to ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
download: s3://sagemaker-us-east-1-1763029

In [146]:
# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]



In [2]:
import IPython

IPython.display.HTML(filename="ProfilerReport/profiler-output/profiler-report.html")

## Model Deploying

In [101]:
# TODO: Deploy your model to an endpoint

# TODO: Add your deployment configuration like instance type and number of instances
predictor=estimator.deploy(initial_instance_count=1, instance_type='ml.m5.large') 


------!

In [8]:
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import Predictor

In [9]:
jpeg_serializer = sagemaker.serializers.IdentitySerializer("image/jpeg")
json_deserializer = sagemaker.deserializers.JSONDeserializer()


class ImagePredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(ImagePredictor, self).__init__(
            endpoint_name,
            sagemaker_session=sagemaker_session,
            serializer=jpeg_serializer,
            deserializer=json_deserializer,
        )

In [ ]:
model_location=estimator.model_data
model_location

In [3]:
#for debugging after the instance was shut down and reloaded the next day, without restarting an estimator instance
model_location = 's3://sagemaker-us-east-1-176302953206/pytorch-training-2022-01-12-14-35-02-261/output/model.tar.gz'

In [11]:
pytorch_model = PyTorchModel(model_data=model_location, role=role, entry_point='inference.py',py_version='py3',
                             framework_version='1.4',
                             predictor_cls=ImagePredictor)

In [19]:
deployment = pytorch_model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

----------!

In [21]:
endpoint = deployment.endpoint_name
endpoint

'pytorch-inference-2022-01-13-14-33-54-650'

In [22]:
predictor = Predictor(endpoint)

In [23]:
##
## INFERENCE : I get an error (likely I called the invoke endpoint too many times, I don't know where the problem comes from)
## 

In [24]:
from sagemaker.serializers import IdentitySerializer
import base64

predictor.serializer = IdentitySerializer("image/png")
with open("./dogImages/test/003.Airedale_terrier/Airedale_terrier_00179.jpg", "rb") as f:
    payload = f.read()

inference = predictor.predict(payload)
inference


InternalFailure: An error occurred (InternalFailure) when calling the InvokeEndpoint operation (reached max retries: 4): An exception occurred while sending request to model. Please contact customer support regarding request 78bac4d9-dadf-4aa4-b544-df43fa25832a.

In [25]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()